In [142]:
N,pxl = 25, 200
# X,Y,R = -0.151606, 0.907705, 0.000001
X = -0.04524074130409
Y = 0.9868162207157852
R = 6.8e-14
ranges = np.zeros([N,4])
mandelbrot_sets = np.zeros([N-1,pxl,pxl])

for i in range(N):
    ranges[i] = X-R**(i/N),X+R**(i/N),Y-R**(i/N),Y+R**(i/N) 

for i in range(N-1):
    x0,x1,y0,y1 = ranges[i]
    x0_new,x1_new,y0_new,y1_new = ranges[i+1]
    x_range = np.linspace(x0,x1,pxl)
    y_range = np.linspace(y0,y1,pxl)[::-1]

    mandelbrot_set = np.zeros([pxl,pxl])
    for m,x in enumerate(x_range):
        if m%50==0:
            print(i,m)
        for n,y in enumerate(y_range):
            c = x+y*1j
            z = 0
            count = 0
            while np.abs(z)<2:
                z = z**2 + c
                count += 1
                if count > 100:
                    break
            mandelbrot_set[m][n] = count
            if count > 100:
                mandelbrot_set[m][n] = 0   
    
    mandelbrot_sets[i] = np.transpose(mandelbrot_set)




0 0
0 50
0 100
0 150
1 0
1 50
1 100
1 150
2 0
2 50
2 100
2 150
3 0
3 50
3 100
3 150
4 0
4 50
4 100
4 150
5 0
5 50
5 100
5 150
6 0
6 50
6 100
6 150
7 0
7 50
7 100
7 150
8 0
8 50
8 100
8 150
9 0
9 50
9 100
9 150
10 0
10 50
10 100
10 150
11 0
11 50
11 100
11 150
12 0
12 50
12 100
12 150
13 0
13 50
13 100
13 150
14 0
14 50
14 100
14 150
15 0
15 50
15 100
15 150
16 0
16 50
16 100
16 150
17 0
17 50
17 100
17 150
18 0
18 50
18 100
18 150
19 0
19 50
19 100
19 150
20 0
20 50
20 100
20 150
21 0
21 50
21 100
21 150
22 0
22 50
22 100
22 150
23 0
23 50
23 100
23 150


In [143]:
from ipywidgets import interact, IntSlider

def animate(i):
    plt.imshow(mandelbrot_sets[i])

interact(animate, i = IntSlider(min=0, max=N-2, value=0))

interactive(children=(IntSlider(value=0, description='i', max=23), Output()), _dom_classes=('widget-interact',…

<function __main__.animate(i)>

In [141]:
np.max(mandelbrot_sets[30])

0.0